In [ ]:
!pip install mlflow

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import mlflow
import mlflow.pytorch
from torchvision import models


mlflow.set_experiment = 'example1'

In [ ]:

def load_mnist(batch_size=64):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
    test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader


def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    return total_loss, accuracy


def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    return total_loss, accuracy

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, test_loader = load_mnist()


    class Net(nn.Module):
      def __init__(self):
          super(Net, self).__init__()
          self.conv1 = nn.Conv2d(1, 6, 5)
          self.pool = nn.MaxPool2d(2, 2)
          self.conv2 = nn.Conv2d(6, 16, 5)
          self.fc1 = nn.Linear(16 * 4 * 4, 120)
          self.fc2 = nn.Linear(120, 84)
          self.fc3 = nn.Linear(84, 10)

      def forward(self, x):
          x = self.pool(torch.relu(self.conv1(x)))
          x = self.pool(torch.relu(self.conv2(x)))
          x = x.view(-1, 16 * 4 * 4)
          x = torch.relu(self.fc1(x))
          x = torch.relu(self.fc2(x))
          x = self.fc3(x)
          return x

    model = Net().to(device)

    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    with mlflow.start_run():
        mlflow.set_tag("model_name", "ResNet")
        mlflow.log_params({"learning_rate": 0.01, "momentum": 0.9})

        for epoch in range(10):
            train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)
            test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)


            mlflow.log_metric("train_loss", train_loss)
            mlflow.log_metric("train_accuracy", train_accuracy)
            mlflow.log_metric("test_loss", test_loss)
            mlflow.log_metric("test_accuracy", test_accuracy)

            print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Train Accuracy={train_accuracy:.2f}%, "
                  f"Test Loss={test_loss:.4f}, Test Accuracy={test_accuracy:.2f}%")


        mlflow.pytorch.log_model(model, "mnist_classification_model")


NameError: ignored

In [ ]:
import shutil

shutil.make_archive("/content/mlruns", 'zip', "/content/mlruns")

'/content/mlruns.zip'

In [ ]:
from google.colab import files

files.download("/content/mlruns.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>